# TMDB에 맞춰 데이터 수정

In [4]:
import pandas as pd

final_metadata = pd.read_excel('final_metadata_with_overview_embeddings_ver4_0510.xlsx')

In [42]:
import requests

def get_movie(title, year=None):
    url = f"https://api.themoviedb.org/3/search/movie?query={title}&include_adult=false&language=en&page=1&year={year}"

    headers = {
        "accept": "application/json",
        "Authorization": "Bearer "
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        return None
    
    data = response.json()
    # print(data['results'][0])
    if len(data['results']) == 0:
        return None
    
    for result in data['results']:
        # title_in_result = result['original_title'].lower() # original_title로 체크
        title_in_result = result['title'].lower()
        title = title.lower()
        if title == title_in_result:
            return result
    return data['results'][0]

get_movie("In Our Prime")

{'adult': False,
 'backdrop_path': '/bDFI9ix7k6FAeZ0O4urYuY3XNzs.jpg',
 'genre_ids': [18],
 'id': 628802,
 'original_language': 'ko',
 'original_title': '이상한 나라의 수학자',
 'overview': 'Ji-woo, an outcast at a prestigious private school, meets Hak-sung, the school janitor who is actually a mathematical genius who defected from North Korea.',
 'popularity': 6.998,
 'poster_path': '/ypUHhgPyawYu2SeTHMmaYUZ9HaF.jpg',
 'release_date': '2022-03-18',
 'title': 'In Our Prime',
 'video': False,
 'vote_average': 8.0,
 'vote_count': 48}

In [43]:
import time

tmdb_data = pd.DataFrame(columns=['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date', 'title', 'video', 'vote_average', 'vote_count'])
check = pd.DataFrame()

for i in range(len(final_metadata)):
    # print(i)
    # title = final_metadata['originalTitle'][i] # originalTitle
    title = final_metadata['primaryTitle'][i]
    year = final_metadata['startYear'][i]
    data = get_movie(title, year)
    if data:
        # final_metadata['overview'][i] = data['overview']
        # 만약 data['genre_ids']가 list면 아래와 같이 변경
        if type(data['genre_ids']) == list:
            data['genre_ids'] = ','.join(map(str, data['genre_ids']))
        tmdb_data = pd.concat([tmdb_data, pd.DataFrame(data, index=[0])], ignore_index=True)
    else:
        print(f"Failed to get overview for {title}")

    # 1000회 요청 시 3초 쉬기
    if i % 1000 == 0:
        time.sleep(3)
tmdb_data

/var/folders/vw/3wnj_h7s68l_wbf3ytj2csd80000gn/T/ipykernel_69377/1541005140.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmdb_data = pd.concat([tmdb_data, pd.DataFrame(data, index=[0])], ignore_index=True)


Failed to get overview for Farewell My Concubine
Failed to get overview for Feng quan dian tui


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/u3JuBKQTuyfvWy7Rm01Ugej6Qs5.jpg,18,1146410,ko,여행자의 필요,"A French woman, who initially played a child's...",7.107,/j74qm3AmNzjZ0Qxeyich6dY2SsT.jpg,2024-04-24,A Traveler's Needs,False,0.000,0
1,False,/obJgmXo6wYEYs8OuIvRgYhx7tHx.jpg,"18,35",876127,ko,도그데이즈,The story of people connected through dogs at ...,5.918,/qGVzvsB34oR7MAJmO2P4DFOegjR.jpg,2024-02-07,Dog Days,False,6.800,4
2,False,/klb3yODwKMRle19EcvdeSatEZL7.jpg,27,1072342,en,Night Swim,Forced into early retirement by a degenerative...,84.375,/gSkfBGdxdialBMM7P02V4hcI6Ij.jpg,2024-01-03,Night Swim,False,5.700,399
3,False,/aINel9503ompOlGKn4sIVMg09Un.jpg,"9648,27,53",838209,ko,파묘,After tracing the origin of a disturbing super...,190.952,/pQYHouPsDw32FhDLr7E3jmw0WTk.jpg,2024-02-22,Exhuma,False,7.500,90
4,False,/3DPJtqKKMl3Lv8rMxwqNLlRc947.jpg,"80,28,18",1017163,ko,범죄도시 4,'Monster Cop' Ma Seok-do investigates an illeg...,87.573,/yk38mNoJpsswmk9o7i7eLhO4mc.jpg,2024-04-24,The Roundup: Punishment,False,8.600,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9518,False,/3Ztq49qcwnXSC9JFmTuaLNYEGKX.jpg,35,67319,en,So This Is Paris,Paul and Suzanne Giraud are happily married an...,6.907,/bVcYRrzmA95SIlPVd1RGGrd17XA.jpg,1926-07-31,So This Is Paris,False,6.500,19
9519,False,/sTyt7e2PB5nfY02reL1iZUYPbly.jpg,18,5991,de,Der letzte Mann,"An aging doorman, after being fired from his p...",14.364,/7Y6Dxr3oYt1w7ew70YNdLLDYEjk.jpg,1924-12-23,The Last Laugh,False,7.785,288
9520,False,/y8WOKk49VdiA4RZ1UeFCt8xPVTF.jpg,"18,10749",28974,en,A Woman of Paris: A Drama of Fate,When Marie St. Clair believes she has been jil...,12.706,/s24VgyvJliT7fUA909ZHJpoBx1D.jpg,1923-10-01,A Woman of Paris: A Drama of Fate,False,6.731,158
9521,False,/bKNfpShviRHQAFwruRpqjj6iqxC.jpg,18,175472,de,Vier um die Frau,Harry Yquem buys his beloved wife some jewelry...,5.730,/bvBfm9HFOrjmXoqPLa9aeDgCnSA.jpg,1921-02-03,Four Around the Woman,False,5.800,15


## 누락된 데이터 직접 추가

In [44]:
data = get_movie("Farewell My Concubine", 1993)
if data:
    # final_metadata['overview'][i] = data['overview']
    # 만약 data['genre_ids']가 list면 아래와 같이 변경
    if type(data['genre_ids']) == list:
        data['genre_ids'] = ','.join(map(str, data['genre_ids']))
        movie_data = pd.DataFrame(data, index=[0])
    # movie_data를 tmdb_data의 7794 행으로 삽입
    tmdb_data = pd.concat([tmdb_data.iloc[:7794], movie_data, tmdb_data.iloc[7794:]], ignore_index=True)
else:
    print(f"Failed to get overview for {title}")

data = get_movie("Iron Fisted Eagle's Claw", 1979)
if data:
    # final_metadata['overview'][i] = data['overview']
    # 만약 data['genre_ids']가 list면 아래와 같이 변경
    if type(data['genre_ids']) == list:
        data['genre_ids'] = ','.join(map(str, data['genre_ids']))
        movie_data = pd.DataFrame(data, index=[0])
    # movie_data를 tmdb_data의 8546 행으로 삽입
    tmdb_data = pd.concat([tmdb_data.iloc[:8546], movie_data, tmdb_data.iloc[8546:]], ignore_index=True)
else:
    print(f"Failed to get overview for {title}")

In [45]:
tmdb_data

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/u3JuBKQTuyfvWy7Rm01Ugej6Qs5.jpg,18,1146410,ko,여행자의 필요,"A French woman, who initially played a child's...",7.107,/j74qm3AmNzjZ0Qxeyich6dY2SsT.jpg,2024-04-24,A Traveler's Needs,False,0.000,0
1,False,/obJgmXo6wYEYs8OuIvRgYhx7tHx.jpg,"18,35",876127,ko,도그데이즈,The story of people connected through dogs at ...,5.918,/qGVzvsB34oR7MAJmO2P4DFOegjR.jpg,2024-02-07,Dog Days,False,6.800,4
2,False,/klb3yODwKMRle19EcvdeSatEZL7.jpg,27,1072342,en,Night Swim,Forced into early retirement by a degenerative...,84.375,/gSkfBGdxdialBMM7P02V4hcI6Ij.jpg,2024-01-03,Night Swim,False,5.700,399
3,False,/aINel9503ompOlGKn4sIVMg09Un.jpg,"9648,27,53",838209,ko,파묘,After tracing the origin of a disturbing super...,190.952,/pQYHouPsDw32FhDLr7E3jmw0WTk.jpg,2024-02-22,Exhuma,False,7.500,90
4,False,/3DPJtqKKMl3Lv8rMxwqNLlRc947.jpg,"80,28,18",1017163,ko,범죄도시 4,'Monster Cop' Ma Seok-do investigates an illeg...,87.573,/yk38mNoJpsswmk9o7i7eLhO4mc.jpg,2024-04-24,The Roundup: Punishment,False,8.600,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9520,False,/3Ztq49qcwnXSC9JFmTuaLNYEGKX.jpg,35,67319,en,So This Is Paris,Paul and Suzanne Giraud are happily married an...,6.907,/bVcYRrzmA95SIlPVd1RGGrd17XA.jpg,1926-07-31,So This Is Paris,False,6.500,19
9521,False,/sTyt7e2PB5nfY02reL1iZUYPbly.jpg,18,5991,de,Der letzte Mann,"An aging doorman, after being fired from his p...",14.364,/7Y6Dxr3oYt1w7ew70YNdLLDYEjk.jpg,1924-12-23,The Last Laugh,False,7.785,288
9522,False,/y8WOKk49VdiA4RZ1UeFCt8xPVTF.jpg,"18,10749",28974,en,A Woman of Paris: A Drama of Fate,When Marie St. Clair believes she has been jil...,12.706,/s24VgyvJliT7fUA909ZHJpoBx1D.jpg,1923-10-01,A Woman of Paris: A Drama of Fate,False,6.731,158
9523,False,/bKNfpShviRHQAFwruRpqjj6iqxC.jpg,18,175472,de,Vier um die Frau,Harry Yquem buys his beloved wife some jewelry...,5.730,/bvBfm9HFOrjmXoqPLa9aeDgCnSA.jpg,1921-02-03,Four Around the Woman,False,5.800,15


## 잘못된 데이터 직접 수정

In [46]:
tmdb_data.loc[tmdb_data["title"] == "The Wait"]

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
2042,False,None,18,1075494,en,The Wait,,4.047,/5UAXRGlyhteJC93wKd8vxilymiH.jpg,2021-04-30,The Wait,False,0.0,0


In [47]:
# 아래 정보가 진짜 영화 The Wait의 정보
new_data = pd.DataFrame([['False', '/tuJMYSFTOqt3QYq0BuZx8Lmeiq3.jpg', '10749,18', 716814, 'fi', 'Odotus', 'The idyllic and secluded Finnish archipelago. The pandemic soars across the world. Elli’s world trembles as her husband’s old friend arrives at the island.', 28.534, '/e5qhyVvLkK5e0B5XTvThrSzNyJf.jpg', '2022-04-01', 'The Wait', 'False', 6.0, 21]], columns=tmdb_data.columns)
tmdb_data.loc[tmdb_data["title"] == "The Wait"] = new_data.values[0]
tmdb_data.iloc[2042,:]

adult                                                            False
backdrop_path                         /tuJMYSFTOqt3QYq0BuZx8Lmeiq3.jpg
genre_ids                                                     10749,18
id                                                              716814
original_language                                                   fi
original_title                                                  Odotus
overview             The idyllic and secluded Finnish archipelago. ...
popularity                                                      28.534
poster_path                           /e5qhyVvLkK5e0B5XTvThrSzNyJf.jpg
release_date                                                2022-04-01
title                                                         The Wait
video                                                            False
vote_average                                                       6.0
vote_count                                                          21
Name: 

In [48]:
tmdb_data.to_csv('tmdb_data_0511_9525movies.csv', index=False)

# check

In [14]:
final_metadata = pd.read_excel('final_metadata_with_overview_embeddings_ver4_0510.xlsx')
final_metadata

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,ordering,...,embedding_374,embedding_375,embedding_376,embedding_377,embedding_378,embedding_379,embedding_380,embedding_381,embedding_382,embedding_383
0,tt31015344,A Traveler's Needs,Yeohaengjaui Pilyo,2024,90,Drama,6.5,75,nm0393254,1,...,0.016458,-0.023006,-0.050298,0.037739,-0.076090,-0.010432,-0.058701,0.017649,0.035267,-0.023228
1,tt30911335,Dog Days,Dogeudeijeu,2024,120,Drama,7.8,31,nm15730187,1,...,-0.034701,-0.008065,-0.024117,-0.003312,0.038821,-0.030186,0.019926,0.074339,-0.018989,0.055851
2,tt9682428,Night Swim,Night Swim,2024,98,"Horror,Thriller",4.7,12654,nm3241399,1,...,0.003738,-0.049932,-0.121888,-0.054388,-0.020564,0.050116,0.021484,0.052849,0.035944,0.025606
3,tt27802490,Exhuma,Pamyo,2024,134,"Horror,Mystery,Thriller",7.4,495,nm6940206,1,...,0.051392,-0.002096,0.044035,0.059909,-0.044593,-0.032423,0.081002,0.020807,0.021268,0.008938
4,tt27811040,The Roundup: Punishment,Beomjoedosi4,2024,109,"Action,Crime,Thriller",8.8,101,nm2441168,10,...,0.041042,-0.039577,0.034051,0.041895,-0.132000,-0.051895,0.089292,-0.132800,-0.011520,-0.006363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9520,tt0017409,So This Is Paris,So This Is Paris,1926,60,Comedy,7.1,746,nm0523932,1,...,-0.052198,-0.056215,-0.065513,0.068392,0.040504,-0.013506,-0.068042,0.001189,-0.021429,-0.103460
9521,tt0015064,The Last Laugh,Der letzte Mann,1924,88,Drama,8.0,15105,nm0003638,1,...,0.029271,-0.052846,0.032112,0.051352,-0.038684,-0.013877,0.074433,-0.020061,-0.008468,0.026134
9522,tt0014624,A Woman of Paris: A Drama of Fate,A Woman of Paris: A Drama of Fate,1923,82,"Drama,Romance",6.9,6104,nm0000122,10,...,0.018064,-0.050176,-0.045593,0.036943,-0.046362,-0.033189,-0.061279,-0.036766,0.073398,-0.105024
9523,tt0012806,Four Around the Woman,Vier um die Frau,1921,52,Drama,5.9,581,nm0000485,1,...,-0.047176,-0.100745,0.056739,0.063528,-0.031887,-0.028223,-0.002628,-0.041470,0.026425,0.023838


In [ ]:
final_data = pd.read_excel("final_metadata_with_overview_embeddings_ver4_0510.xlsx")
final_data

real_final_data = final_data.iloc[:, :-384].copy()
real_final_data

from sentence_transformers import SentenceTransformer
import torch

# 모델 로딩
model = SentenceTransformer('all-MiniLM-L6-v2')

# 영화 개요를 임베딩 벡터로 변환
embeddings = model.encode(tmdb_data['overview'].tolist())

# 임베딩 결과 확인
print(embeddings)

# 임베딩 벡터를 리스트화하지 않고 각 값이 하나의 feature가 되도록 차원 수에 맞게 column을 추가하고 데이터프레임에 추가
for i in range(384):
    real_final_data[f'embedding_{i}'] = [x[i] for x in embeddings]

In [ ]:
real_final_data.to_excel("final_metadata_with_overview_embeddings_ver5_0511.xlsx")

## 처리한 데이터와 과거에 tmdb에서 가져온 줄거리 정보가 일치하는지 확인

In [50]:
tmdb_ver2 = pd.read_csv('tmdb_data_ver2.csv')
tmdb_ver2

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/u3JuBKQTuyfvWy7Rm01Ugej6Qs5.jpg,18,1146410,ko,여행자의 필요,"A French woman, who initially played a child's...",6.263,/j74qm3AmNzjZ0Qxeyich6dY2SsT.jpg,2024-04-24,A Traveler's Needs,False,0.000,0
1,False,/obJgmXo6wYEYs8OuIvRgYhx7tHx.jpg,"18,35",876127,ko,도그데이즈,The story of people connected through dogs at ...,4.271,/qGVzvsB34oR7MAJmO2P4DFOegjR.jpg,2024-02-07,Dog Days,False,6.800,4
2,False,/klb3yODwKMRle19EcvdeSatEZL7.jpg,"27,53",1072342,en,Night Swim,Forced into early retirement by a degenerative...,81.423,/gSkfBGdxdialBMM7P02V4hcI6Ij.jpg,2024-01-03,Night Swim,False,5.740,359
3,False,/aINel9503ompOlGKn4sIVMg09Un.jpg,"9648,27,53",838209,ko,파묘,After tracing the origin of a disturbing super...,252.396,/pQYHouPsDw32FhDLr7E3jmw0WTk.jpg,2024-02-22,Exhuma,False,7.800,38
4,False,/mO9oOVXM8tTlC11VFM4FBBNnL3f.jpg,"80,28,18",1017163,ko,범죄도시 4,'Monster Cop' Ma Seok-do investigates an illeg...,34.932,/r7LHM9MsPEipFCBPbsV6nHzSw3D.jpg,2024-04-24,The Roundup: Punishment,False,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9696,False,/3Ztq49qcwnXSC9JFmTuaLNYEGKX.jpg,35,67319,en,So This Is Paris,Paul and Suzanne Giraud are happily married an...,3.019,/bVcYRrzmA95SIlPVd1RGGrd17XA.jpg,1926-07-31,So This Is Paris,False,6.500,19
9697,False,/sTyt7e2PB5nfY02reL1iZUYPbly.jpg,18,5991,de,Der letzte Mann,"An aging doorman, after being fired from his p...",10.232,/7Y6Dxr3oYt1w7ew70YNdLLDYEjk.jpg,1924-12-23,The Last Laugh,False,7.800,287
9698,False,/y8WOKk49VdiA4RZ1UeFCt8xPVTF.jpg,"18,10749",28974,en,A Woman of Paris: A Drama of Fate,When Marie St. Clair believes she has been jil...,10.437,/s24VgyvJliT7fUA909ZHJpoBx1D.jpg,1923-10-01,A Woman of Paris: A Drama of Fate,False,6.700,156
9699,False,/bKNfpShviRHQAFwruRpqjj6iqxC.jpg,18,175472,de,Vier um die Frau,Harry Yquem buys his beloved wife some jewelry...,2.633,/bvBfm9HFOrjmXoqPLa9aeDgCnSA.jpg,1921-02-03,Four Around the Woman,False,5.800,15


### tmdb_ver2를 tmdb_data와 같은 영화 순서로 정렬하여 줄거리를 비교

In [53]:
# tmdb_ver2를 tmdb_data와 같은 영화 순서로 정렬하여 줄거리를 비교
# tmdb_ver2의 9701개의 데이터 중 tmdb_data와 일치하는 것만 남김
tmdb_ver2_after_filtering = tmdb_ver2.loc[tmdb_ver2["title"].isin(tmdb_data["title"])].reset_index(drop=True)
tmdb_ver2_after_filtering

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/u3JuBKQTuyfvWy7Rm01Ugej6Qs5.jpg,18,1146410,ko,여행자의 필요,"A French woman, who initially played a child's...",6.263,/j74qm3AmNzjZ0Qxeyich6dY2SsT.jpg,2024-04-24,A Traveler's Needs,False,0.000,0
1,False,/obJgmXo6wYEYs8OuIvRgYhx7tHx.jpg,"18,35",876127,ko,도그데이즈,The story of people connected through dogs at ...,4.271,/qGVzvsB34oR7MAJmO2P4DFOegjR.jpg,2024-02-07,Dog Days,False,6.800,4
2,False,/klb3yODwKMRle19EcvdeSatEZL7.jpg,"27,53",1072342,en,Night Swim,Forced into early retirement by a degenerative...,81.423,/gSkfBGdxdialBMM7P02V4hcI6Ij.jpg,2024-01-03,Night Swim,False,5.740,359
3,False,/aINel9503ompOlGKn4sIVMg09Un.jpg,"9648,27,53",838209,ko,파묘,After tracing the origin of a disturbing super...,252.396,/pQYHouPsDw32FhDLr7E3jmw0WTk.jpg,2024-02-22,Exhuma,False,7.800,38
4,False,/mO9oOVXM8tTlC11VFM4FBBNnL3f.jpg,"80,28,18",1017163,ko,범죄도시 4,'Monster Cop' Ma Seok-do investigates an illeg...,34.932,/r7LHM9MsPEipFCBPbsV6nHzSw3D.jpg,2024-04-24,The Roundup: Punishment,False,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9512,False,/3Ztq49qcwnXSC9JFmTuaLNYEGKX.jpg,35,67319,en,So This Is Paris,Paul and Suzanne Giraud are happily married an...,3.019,/bVcYRrzmA95SIlPVd1RGGrd17XA.jpg,1926-07-31,So This Is Paris,False,6.500,19
9513,False,/sTyt7e2PB5nfY02reL1iZUYPbly.jpg,18,5991,de,Der letzte Mann,"An aging doorman, after being fired from his p...",10.232,/7Y6Dxr3oYt1w7ew70YNdLLDYEjk.jpg,1924-12-23,The Last Laugh,False,7.800,287
9514,False,/y8WOKk49VdiA4RZ1UeFCt8xPVTF.jpg,"18,10749",28974,en,A Woman of Paris: A Drama of Fate,When Marie St. Clair believes she has been jil...,10.437,/s24VgyvJliT7fUA909ZHJpoBx1D.jpg,1923-10-01,A Woman of Paris: A Drama of Fate,False,6.700,156
9515,False,/bKNfpShviRHQAFwruRpqjj6iqxC.jpg,18,175472,de,Vier um die Frau,Harry Yquem buys his beloved wife some jewelry...,2.633,/bvBfm9HFOrjmXoqPLa9aeDgCnSA.jpg,1921-02-03,Four Around the Woman,False,5.800,15


# 필요한 데이터 전체(한국어로) 데이터 가져오기

In [16]:
tmdb_data = pd.read_csv('tmdb_data_0511_9525movies.csv')
tmdb_data

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/u3JuBKQTuyfvWy7Rm01Ugej6Qs5.jpg,18,1146410,ko,여행자의 필요,"A French woman, who initially played a child's...",7.107,/j74qm3AmNzjZ0Qxeyich6dY2SsT.jpg,2024-04-24,A Traveler's Needs,False,0.000,0
1,False,/obJgmXo6wYEYs8OuIvRgYhx7tHx.jpg,"18,35",876127,ko,도그데이즈,The story of people connected through dogs at ...,5.918,/qGVzvsB34oR7MAJmO2P4DFOegjR.jpg,2024-02-07,Dog Days,False,6.800,4
2,False,/klb3yODwKMRle19EcvdeSatEZL7.jpg,27,1072342,en,Night Swim,Forced into early retirement by a degenerative...,84.375,/gSkfBGdxdialBMM7P02V4hcI6Ij.jpg,2024-01-03,Night Swim,False,5.700,399
3,False,/aINel9503ompOlGKn4sIVMg09Un.jpg,"9648,27,53",838209,ko,파묘,After tracing the origin of a disturbing super...,190.952,/pQYHouPsDw32FhDLr7E3jmw0WTk.jpg,2024-02-22,Exhuma,False,7.500,90
4,False,/3DPJtqKKMl3Lv8rMxwqNLlRc947.jpg,"80,28,18",1017163,ko,범죄도시 4,'Monster Cop' Ma Seok-do investigates an illeg...,87.573,/yk38mNoJpsswmk9o7i7eLhO4mc.jpg,2024-04-24,The Roundup: Punishment,False,8.600,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9520,False,/3Ztq49qcwnXSC9JFmTuaLNYEGKX.jpg,35,67319,en,So This Is Paris,Paul and Suzanne Giraud are happily married an...,6.907,/bVcYRrzmA95SIlPVd1RGGrd17XA.jpg,1926-07-31,So This Is Paris,False,6.500,19
9521,False,/sTyt7e2PB5nfY02reL1iZUYPbly.jpg,18,5991,de,Der letzte Mann,"An aging doorman, after being fired from his p...",14.364,/7Y6Dxr3oYt1w7ew70YNdLLDYEjk.jpg,1924-12-23,The Last Laugh,False,7.785,288
9522,False,/y8WOKk49VdiA4RZ1UeFCt8xPVTF.jpg,"18,10749",28974,en,A Woman of Paris: A Drama of Fate,When Marie St. Clair believes she has been jil...,12.706,/s24VgyvJliT7fUA909ZHJpoBx1D.jpg,1923-10-01,A Woman of Paris: A Drama of Fate,False,6.731,158
9523,False,/bKNfpShviRHQAFwruRpqjj6iqxC.jpg,18,175472,de,Vier um die Frau,Harry Yquem buys his beloved wife some jewelry...,5.730,/bvBfm9HFOrjmXoqPLa9aeDgCnSA.jpg,1921-02-03,Four Around the Woman,False,5.800,15


In [23]:
# cinemate_data = pd.DataFrame(columns=[
#     'movie_id', # movieId
#     'rating', # rating
#     'backdrop_path', # backdropPath
#     'original_title', # originalTitle
#     'title', # movieTitle
#     'release_date',  # releaseDate
#     'poster_path', # posterPath
#     'overview' # overview
#     ])

In [29]:
# cinemate_data = pd.concat([cinemate_data, pd.DataFrame({
#     'movie_id': data['id'],
#     'rating': data['vote_average'],
#     'backdrop_path': data['backdrop_path'],
#     'original_title': data['original_title'],
#     'title': data['title'],
#     'release_date': data['release_date'],
#     'poster_path': data['poster_path'],
#     'overview': data['overview']
# }, index=[0])], ignore_index=True)
# cinemate_data

,movie_id,rating,backdrop_path,original_title,title,release_date,poster_path,overview
0,876127,6.8,/obJgmXo6wYEYs8OuIvRgYhx7tHx.jpg,도그데이즈,도그데이즈,2024-02-07,/bl3YQHEENj4B80Ku3PkTJWWLnj5.jpg,깔끔한 성격의 계획형 싱글남 민상. 영끌까지 모아 산 건물을 개똥밭으로 만드는 세입...
1,838209,7.5,/aINel9503ompOlGKn4sIVMg09Un.jpg,파묘,파묘,2024-02-22,/tw0i3kkmOTjDjGFZTLHKhoeXVvA.jpg,"미국 LA, 거액의 의뢰를 받은 무당 화림과 봉길은 기이한 병이 대물림되는 집안의 ..."


## 기본 정보

In [31]:
import requests

def get_movie_info_ko(movie_id):
    url_for_main_info = f"https://api.themoviedb.org/3/movie/{movie_id}?language=ko-KR"

    headers = {
        "accept": "application/json",
        "Authorization": "Bearer "
    }

    main_info_response = requests.get(url_for_main_info, headers=headers)

    if main_info_response.status_code != 200:
        print("Failed to get movie info")
        return None
    
    data = main_info_response.json()
    
    return data


get_movie_info_ko(838209)

{'adult': False,
 'backdrop_path': '/aINel9503ompOlGKn4sIVMg09Un.jpg',
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 9648, 'name': '미스터리'},
  {'id': 27, 'name': '공포'},
  {'id': 53, 'name': '스릴러'}],
 'homepage': '',
 'id': 838209,
 'imdb_id': 'tt27802490',
 'origin_country': ['KR'],
 'original_language': 'ko',
 'original_title': '파묘',
 'overview': '미국 LA, 거액의 의뢰를 받은 무당 화림과 봉길은 기이한 병이 대물림되는 집안의 장손을 만난다. 조상의 묫자리가 화근임을 알아챈 화림은 이장을 권하고, 돈 냄새를 맡은 최고의 풍수사 상덕과 장의사 영근이 합류한다. 절대 사람이 묻힐 수 없는 악지에 자리한 기이한 묘. 상덕은 불길한 기운을 느끼고 제안을 거절하지만, 화림의 설득으로 결국 파묘가 시작되고… 나와서는 안될 것이 나왔다.',
 'popularity': 190.952,
 'poster_path': '/tw0i3kkmOTjDjGFZTLHKhoeXVvA.jpg',
 'production_companies': [{'id': 3491,
   'logo_path': '/hXRKXi3M9qfP4Npi1kjAaH4eGnM.png',
   'name': 'Showbox',
   'origin_country': 'KR'},
  {'id': 15974,
   'logo_path': '/whf327LIlsbElBmCXIgnL0aueko.png',
   'name': 'MCMC',
   'origin_country': 'KR'},
  {'id': 217380,
   'logo_path': None,
   'name': 'Pinetown Production',
   'orig

In [32]:
cinemate_data = pd.DataFrame(columns=[
    'movie_id', # movieId
    'rating', # rating
    'backdrop_path', # backdropPath
    'original_title', # originalTitle
    'title', # movieTitle
    'release_date',  # releaseDate
    'poster_path', # posterPath
    'overview' # overview
    ])

for i in range(len(tmdb_data)):
    movie_id = tmdb_data['id'][i]
    data = get_movie_info_ko(movie_id)
    if data:
        cinemate_data = pd.concat([cinemate_data, pd.DataFrame({
            'movie_id': data['id'],
            'rating': data['vote_average'],
            'backdrop_path': data['backdrop_path'],
            'original_title': data['original_title'],
            'title': data['title'],
            'release_date': data['release_date'],
            'poster_path': data['poster_path'],
            'overview': data['overview']
        }, index=[0])], ignore_index=True)
    else:
        print(f"Failed to get movie info for {movie_id}")

/var/folders/vw/3wnj_h7s68l_wbf3ytj2csd80000gn/T/ipykernel_42130/2436752402.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cinemate_data = pd.concat([cinemate_data, pd.DataFrame({


In [33]:
cinemate_data

,movie_id,rating,backdrop_path,original_title,title,release_date,poster_path,overview
0,1146410,0.000,/u3JuBKQTuyfvWy7Rm01Ugej6Qs5.jpg,여행자의 필요,여행자의 필요,2024-04-24,/fNhyJp2fy47vg66LZ8aA2b0tsVQ.jpg,"어디서 온지 모르는 이 사람은 불란서에서 왔다고 하고, 어린애 피리를 근린공원에서 ..."
1,876127,6.800,/obJgmXo6wYEYs8OuIvRgYhx7tHx.jpg,도그데이즈,도그데이즈,2024-02-07,/bl3YQHEENj4B80Ku3PkTJWWLnj5.jpg,깔끔한 성격의 계획형 싱글남 민상. 영끌까지 모아 산 건물을 개똥밭으로 만드는 세입...
2,1072342,5.729,/klb3yODwKMRle19EcvdeSatEZL7.jpg,Night Swim,나이트 스윔,2024-01-03,/4Q20D0HCYNHwvTna4OXJNCDvyI3.jpg,넓은 수영장이 있는 새집으로 이사 온 후 더할 나위 없는 행복을 느끼고 있는 ‘레이...
3,838209,7.500,/aINel9503ompOlGKn4sIVMg09Un.jpg,파묘,파묘,2024-02-22,/tw0i3kkmOTjDjGFZTLHKhoeXVvA.jpg,"미국 LA, 거액의 의뢰를 받은 무당 화림과 봉길은 기이한 병이 대물림되는 집안의 ..."
4,1017163,8.600,/3DPJtqKKMl3Lv8rMxwqNLlRc947.jpg,범죄도시 4,범죄도시 4,2024-04-24,/h1YarEjeYurkAwXgfY1RDMVCiin.jpg,"신종 마약 사건 3년 뒤, 괴물형사 마석도와 서울 광수대는 배달앱을 이용한 마약 판..."
...,...,...,...,...,...,...,...,...
9520,67319,6.500,/3Ztq49qcwnXSC9JFmTuaLNYEGKX.jpg,So This Is Paris,여기는 파리,1926-07-31,/bVcYRrzmA95SIlPVd1RGGrd17XA.jpg,폴과 수자나 부부는 조용한 마을에서 행복하게 살고 있다. 그러던 중 화려한 댄서들이...
9521,5991,7.785,/sTyt7e2PB5nfY02reL1iZUYPbly.jpg,Der letzte Mann,마지막 웃음,1924-12-23,/woR2d4zkWgCPlvwQ9kMHcrdiGPJ.jpg,은발의 멋진 구렛나루 수염을 쓰다듬으며 금빛 단추의 제복을 뽐내는 한 호텔 도어맨이...
9522,28974,6.731,/y8WOKk49VdiA4RZ1UeFCt8xPVTF.jpg,A Woman of Paris: A Drama of Fate,파리의 여인,1923-10-01,/s24VgyvJliT7fUA909ZHJpoBx1D.jpg,마리(Marie St. Clair: 에드나 퍼비안스 분)는 계부의 반대를 피해 애인...
9523,175472,5.800,/bKNfpShviRHQAFwruRpqjj6iqxC.jpg,Vier um die Frau,여자를 둘러싼 넷,1921-02-03,/83nTPJQnzrn78ppVf5L5CEOIkIg.jpg,해리 유켐은 그의 사랑하는 아내에게 가짜와 장물을 거래하는 지하 세계의 중간거래상들...


In [34]:
cinemate_data.to_excel("cinemate_data_0511.xlsx")

## 장르 정보 매핑

In [36]:
final_data = pd.read_excel("final_metadata_with_overview_embeddings_ver5_0511.xlsx")

In [37]:
final_data["genres"]

0                         Drama
1                         Drama
2               Horror,Thriller
3       Horror,Mystery,Thriller
4         Action,Crime,Thriller
                 ...           
9520                     Comedy
9521                      Drama
9522              Drama,Romance
9523                      Drama
9524              Drama,Romance
Name: genres, Length: 9525, dtype: object

In [43]:
for index, row in cinemate_data.iterrows():
    genres = final_data.iloc[index]["genres"]
    cinemate_data.at[index, "genres"] = genres

/var/folders/vw/3wnj_h7s68l_wbf3ytj2csd80000gn/T/ipykernel_42130/2887669575.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Drama' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  cinemate_data.at[index, "genres"] = genres


In [45]:
cinemate_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9525 entries, 0 to 9524
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   movie_id        9525 non-null   object 
 1   rating          9525 non-null   float64
 2   backdrop_path   8791 non-null   object 
 3   original_title  9525 non-null   object 
 4   title           9525 non-null   object 
 5   release_date    9525 non-null   object 
 6   poster_path     9453 non-null   object 
 7   overview        9525 non-null   object 
 8   genres          9525 non-null   object 
dtypes: float64(1), object(8)
memory usage: 669.9+ KB


## SQL 문 생성

In [46]:
genre_mapping = {
    "Action": 1,
    "Adventure": 2,
    "Animation": 3,
    "Biography": 4,
    "Comedy": 5,
    "Crime": 6,
    "Documentary": 7,
    "Drama": 8,
    "Family": 9,
    "Fantasy": 10,
    "History": 11,
    "Horror": 12,
    "Music": 13,
    "Musical": 14,
    "Mystery": 15,
    "Romance": 16,
    "Sci-Fi": 17,
    "Sport": 18,
    "Thriller": 19,
    "War": 20,
    "Western": 21
}

In [55]:
# 영화 데이터 삽입
movie_insert_sql = """
INSERT INTO movie (id, movield, rating, backdropPath, originalTitle, movieTitle, releaseDate, posterPath, overview)
VALUES ({}, '{}', {}, '{}', '{}', '{}', '{}', '{}', '{}');
"""

movies_sql_statements = []

for index, row in cinemate_data.iterrows():
    movie_data = (
        index + 1,
        row['movie_id'],
        row['rating'],
        row['backdrop_path'],
        row['original_title'].replace("'", "''"),
        row['title'].replace("'", "''"),
        row['release_date'],
        row['poster_path'],
        row['overview'].replace("'", "''")
    )
    movie_sql = movie_insert_sql.format(*movie_data)
    movies_sql_statements.append(movie_sql)

# 장르 데이터 삽입
genre_insert_sql = """
INSERT INTO genre (genre_name)
VALUES ('{}');
"""

# genres_sql_statements = []

# # 장르 데이터를 중복 없이 추출
# genres_set = set()
# for index, row in cinemate_data.iterrows():
#     genres = row['genres'].split(',')
#     for genre in genres:
#         genres_set.add(genre.strip())

# # 중복 없는 장르 데이터를 DB에 삽입
# for genre in genres_set:
#     genre_sql = genre_insert_sql.format(genre.strip())
#     genres_sql_statements.append(genre_sql)

# 영화와 장르의 관계 데이터 삽입
movie_genre_mapping_insert_sql = """
INSERT INTO movie_genre_mapping (movie_id, genre_id)
VALUES ({}, {});
"""

movie_genre_sql_statements = []

for index, row in cinemate_data.iterrows():
    genres = row['genres'].split(',')
    for genre in genres:
        genre_id = genre_mapping[genre]
        movie_genre_sql = movie_genre_mapping_insert_sql.format(index + 1, genre_id)
        movie_genre_sql_statements.append(movie_genre_sql)

# SQL 쿼리를 파일에 쓰기
with open('data.sql', 'w') as f:
    for sql_statement in movies_sql_statements + movie_genre_sql_statements:
        f.write(sql_statement + '\n')